In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from pypfopt import EfficientFrontier, risk_models, expected_returns, objective_functions
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import warnings
warnings.filterwarnings("ignore")


In [2]:
data=pd.read_excel("Stock List.xlsx")
data
remove_stock=['OSG','MSTR']
data=data[~data['Ticker'].isin(remove_stock)] 
# data=data[data['Country']=='USA']
# data=data.iloc[:1000]
data.keys()

Index(['Ticker', 'Name', 'Sector', 'Country'], dtype='object')

In [3]:
All_Tickers=data.Ticker.to_list()
All_Tickers = [ticker for ticker in All_Tickers if isinstance(ticker, str)]
sectors = data['Sector'].to_list()


In [4]:
prices_data = yf.download(All_Tickers, start='2023-01-01')['Adj Close']


[***                    6%%                      ]  374 of 6580 completed

Failed to get ticker 'BIO/B' reason: Expecting value: line 1 column 1 (char 0)


[***                    6%%                      ]  385 of 6580 completed

Failed to get ticker 'AKO/B' reason: Expecting value: line 1 column 1 (char 0)


[***                    7%%                      ]  458 of 6580 completed

...

Failed to get ticker 'BRK/A' reason: Expecting value: line 1 column 1 (char 0)


[**********************97%%********************* ]  6415 of 6580 completed

Failed to get ticker 'BF/A' reason: Expecting value: line 1 column 1 (char 0)


[*********************100%%**********************]  6580 of 6580 completed


400 Failed downloads:
['RF^E', 'DBRG^H', 'PSA^J', 'SPG^J', 'VNO^L', 'HPP^C', 'BIP^B', 'TRTN^D', 'INN^F', 'SCE^N', 'AHL^E', 'WFC^L', 'PSEC^A', 'ATCO^D', 'EFC^C', 'FRT^C', 'BFS^D', 'EQC^D', 'BIP^A', 'DRH^A', 'PSA^G', 'NLY^I', 'BIO/B', 'REXR^B', 'RF^B', 'PSA^R', 'AKO/B', 'CFG^D', 'CRD/A', 'C^N', 'MFA^B', 'FBRT^E', 'KEY^L', 'COF^I', 'WBS^F', 'AGM^G', 'WFC^D', 'GL^D', 'RNR^G', 'HL^B', 'GDV^K', 'GLOP^A', 'VNO^N', 'ET^I', 'F^D', 'ACR^D', 'SNV^D', 'ASB^E', 'ACP^A', 'WCC^A', 'XFLT^A', 'PSA^S', 'BAC^B', 'GNL^A', 'DLNG^A', 'NLY^F', 'BHR^D', 'ATCO^H', 'SPLP^A', 'MET^A', 'EQH^A', 'GNL^E', 'F^B', 'NXDT^A', 'MS^E', 'NSA^A', 'SEAL^A', 'CUBI^E', 'EFC^A', 'ACR^C', 'PSA^O', 'GTLS^B', 'NGL^C', 'GS^A', 'MFA^C', 'CIM^C', 'CFG^E', 'INN^E', 'BAC^M', 'CIM^D', 'GS^D', 'NEE^N', 'PEB^F', 'TNP^E', 'DX^C', 'ABR^E', 'JPM^L', 'BC^A', 'PMT^B', 'CIO^A', 'BC^C', 'DUK^A', 'SYF^A', 'JPM^K', 'ALL^B', 'WBS^G', 'TDS^V', 'WFC^C', 'KIM^N', 'BML^H', 'CSR^C', 'WRB^H', 'JPM^D', 'GNL^B', 'GSL^B', 'SNV^E', 'USB^R', 'BML^J', 'SB^D'

In [5]:
prices_data.to_csv("Price_Data.csv")

In [27]:
import datetime
start_date = datetime.datetime.today() - pd.DateOffset(months=6)
prices_train = prices_data.loc[start_date:]
df = prices_train.copy()
df = df.loc[:, ~(df < 1).any()]
# df=df.ffill()
df = df.dropna(axis=1)
df

,A,AA,AACI,AACIU,AADI,AAL,AAME,AAN,AAOI,AAON,...,ZTR,ZTS,ZUMZ,ZUO,ZURA,ZVRA,ZVSA,ZWS,ZYME,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-17 00:00:00,128.119995,27.180000,10.890,10.70,1.59,12.93,2.666404,10.15,16.860001,70.000000,...,5.34,189.926453,17.780001,8.41,3.84,5.760,6.90,28.150000,10.49,9.86
2024-01-18 00:00:00,130.460007,26.750000,10.890,10.70,1.68,13.82,2.626755,10.24,16.620001,70.349998,...,5.31,189.958374,17.629999,8.59,4.03,5.710,5.97,28.500000,9.86,10.34
2024-01-19 00:00:00,131.220001,27.360001,10.890,10.70,1.63,13.66,2.606930,10.24,15.810000,70.320000,...,5.28,187.844406,17.400000,8.57,4.30,5.720,5.50,28.889999,9.88,10.22
2024-01-22 00:00:00,132.550003,27.620001,10.900,10.70,1.68,13.62,2.458246,10.59,17.100000,72.099998,...,5.29,190.656372,17.900000,8.84,4.25,5.970,5.76,29.420000,10.05,10.59
2024-01-23 00:00:00,131.639999,29.510000,10.902,10.70,1.74,14.00,2.478070,10.49,18.129999,70.699997,...,5.24,187.565201,17.799999,9.38,3.98,5.940,6.15,29.170000,10.28,10.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-09 00:00:00,125.419998,40.330002,11.400,11.43,1.39,10.98,1.560000,9.98,8.490000,83.980003,...,5.45,174.240005,20.549999,8.75,3.71,5.475,4.75,28.610001,9.35,8.75
2024-07-10 00:00:00,127.070000,38.459999,11.500,11.43,1.43,11.15,1.560000,9.95,9.010000,85.339996,...,5.47,173.490005,20.590000,8.75,3.65,5.620,6.18,29.180000,9.24,8.98
2024-07-11 00:00:00,130.820007,39.180000,11.480,11.43,1.50,10.73,1.580000,9.96,9.230000,88.570000,...,5.54,177.740005,21.660000,8.99,3.95,6.110,5.05,30.000000,9.87,9.37


In [28]:
# mu = expected_returns.mean_historical_return(df)
# S = risk_models.risk_matrix(df, method='ledoit_wolf')
# ef = EfficientFrontier(mu, S,weight_bounds=(0, 0.4))
# i = 0
# while True:
#     try:
#         raw_weights = ef.efficient_return(target_return=mu.max()-i)
#         print(f"Selected i is {i}")
#         break
#     except Exception as e:
#         # print(f"Error for i={i}: {e}")
#         i += 0.01

# print(f"First successful i: {i}")
# print(f"Raw weights: {raw_weights}")
# # raw_weights=ef.max_sharpe()
# cleaned_weights = ef.clean_weights()
# ef.portfolio_performance(verbose=True)
# filtered_data = [(name, value) for name, value in cleaned_weights.items() if value > 0.1]
# print(filtered_data)

In [29]:
mu = expected_returns.mean_historical_return(df)
S = risk_models.risk_matrix(df, method='ledoit_wolf')

# Initialize Efficient Frontier with sector constraints
sector_info = data.set_index('Ticker')['Sector'].to_dict()
sector_max_allocation = {sector: 0.4 for sector in set(sector_info.values())}
sector_map = {sector: [] for sector in sector_max_allocation.keys()}
for ticker in df.columns:
    sector = sector_info.get(ticker, None)
    if sector:
        sector_map[sector].append(ticker)

In [30]:
sector_map = {sector: [] for sector in sector_max_allocation.keys()}
for ticker in df.columns:
    sector = sector_info.get(ticker, None)
    if sector:
        sector_map[sector].append(ticker)

# Efficient Frontier
ef = EfficientFrontier(mu, S, weight_bounds=(0, 1))

# Add sector constraints directly
for sector, tickers in sector_map.items():
    indices = [df.columns.get_loc(ticker) for ticker in tickers if ticker in df.columns]
    if indices:
        ef.add_constraint(lambda w, indices=indices: sum(w[i] for i in indices) <= sector_max_allocation[sector])

# Optimize portfolio for max Sharpe ratio
# raw_weights = ef.max_sharpe()
i = 0
while True:
    try:
        raw_weights = ef.efficient_return(target_return=mu.max()-i)
        print(f"Selected i is {i}")
        break
    except Exception as e:
        # print(f"Error for i={i}: {e}")
        i += 0.01
# Clean weights and display performance
cleaned_weights = ef.clean_weights()
ef.portfolio_performance(verbose=True)

# Filter non-zero weights
filtered_data = [(name, value) for name, value in cleaned_weights.items() if value > 0.01]
print(filtered_data)

Selected i is 298.8599999998732
Expected annual return: 30040.7%
Annual volatility: 42.5%
Sharpe Ratio: 706.00
[('ACRV', 0.02006), ('BMR', 0.02079), ('BPTH', 0.01386), ('CADL', 0.03084), ('CRBP', 0.02069), ('EBS', 0.03548), ('FENG', 0.01673), ('FTEL', 0.46972), ('HUDA', 0.02407), ('INSG', 0.06485), ('LENZ', 0.0155), ('MMAT', 0.0184), ('PCSA', 0.02225), ('QLI', 0.01822), ('RCRT', 0.01075), ('RDZN', 0.0107), ('SKYE', 0.01004), ('SMR', 0.03832), ('SNPX', 0.01008), ('TSHA', 0.01931), ('WGS', 0.06374), ('WHLR', 0.01896), ('WISA', 0.01163), ('YYAI', 0.03122)]


In [31]:
pd.DataFrame(filtered_data).sort_values(by=1,ascending=False)

,0,1
7,FTEL,0.46972
9,INSG,0.06485
20,WGS,0.06374
17,SMR,0.03832
5,EBS,0.03548
23,YYAI,0.03122
3,CADL,0.03084
8,HUDA,0.02407
12,PCSA,0.02225
1,BMR,0.02079
